In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchsummary import summary

import matplotlib.pyplot as plt

load data 

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [55]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten  = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NN().to(device)
print(model)


NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302351  [   64/60000]
loss: 2.288742  [ 6464/60000]
loss: 2.268091  [12864/60000]
loss: 2.265815  [19264/60000]
loss: 2.253532  [25664/60000]
loss: 2.216333  [32064/60000]
loss: 2.228212  [38464/60000]
loss: 2.193891  [44864/60000]
loss: 2.181459  [51264/60000]
loss: 2.160450  [57664/60000]
Test Error: 
 Accuracy: 40.5%, Avg loss: 2.152003 

Epoch 2
-------------------------------
loss: 2.158452  [   64/60000]
loss: 2.150958  [ 6464/60000]
loss: 2.092839  [12864/60000]
loss: 2.110216  [19264/60000]
loss: 2.066475  [25664/60000]
loss: 1.995857  [32064/60000]
loss: 2.028120  [38464/60000]
loss: 1.948215  [44864/60000]
loss: 1.941154  [51264/60000]
loss: 1.882324  [57664/60000]
Test Error: 
 Accuracy: 59.3%, Avg loss: 1.877513 

Epoch 3
-------------------------------
loss: 1.906847  [   64/60000]
loss: 1.880323  [ 6464/60000]
loss: 1.765201  [12864/60000]
loss: 1.804523  [19264/60000]
loss: 1.701948  [25664/60000]
loss: 1.646124  [32064/600

In [14]:
# check whether the model is on the device
device = next(model.parameters()).device
print(f"Model is on device: {device}")

Model is on device: mps:0


In [23]:
# Temporarily move the model to CPU for summary
model_cpu = model.to("cpu")
# Run summary on the CPU (since torchsummary only supports 'cpu' or 'cuda')
summary(model_cpu, input_size=(1, 28, 28), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
              ReLU-3                  [-1, 512]               0
            Linear-4                  [-1, 512]         262,656
              ReLU-5                  [-1, 512]               0
            Linear-6                   [-1, 10]           5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.55
Estimated Total Size (MB): 2.58
----------------------------------------------------------------


In [46]:
model.named_parameters

<bound method Module.named_parameters of NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)>

In [26]:
# summarize the model parameters
for name, param in model.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter value: {param}")
    print(f"Parameter shape: {param.shape}")
    print(f"Requires gradient: {param.requires_grad}")
    print("-" * 50)

Parameter name: linear_relu_stack.0.weight
Parameter value: Parameter containing:
tensor([[ 0.0198, -0.0240, -0.0281,  ..., -0.0026,  0.0186,  0.0172],
        [-0.0209, -0.0290,  0.0044,  ...,  0.0346, -0.0354,  0.0079],
        [-0.0127, -0.0175, -0.0039,  ..., -0.0025, -0.0199,  0.0257],
        ...,
        [-0.0224,  0.0343, -0.0352,  ...,  0.0286, -0.0338,  0.0129],
        [ 0.0190,  0.0294,  0.0186,  ..., -0.0056, -0.0158,  0.0142],
        [-0.0340, -0.0249, -0.0080,  ...,  0.0003, -0.0034, -0.0322]],
       device='mps:0', requires_grad=True)
Parameter shape: torch.Size([512, 784])
Requires gradient: True
--------------------------------------------------
Parameter name: linear_relu_stack.0.bias
Parameter value: Parameter containing:
tensor([-2.8885e-02,  3.0724e-02,  2.9379e-02,  1.8816e-02, -2.6011e-02,
         3.3328e-03, -8.3296e-03,  2.3787e-02,  1.0857e-02, -1.3107e-02,
        -1.2003e-02, -2.3429e-02,  2.5075e-02, -2.0783e-02,  3.2708e-02,
        -3.0691e-03, -2.002

In [44]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[-0.0066,  0.0215,  0.0193,  ...,  0.0235,  0.0274, -0.0279],
                      [-0.0350, -0.0250, -0.0346,  ..., -0.0334,  0.0346, -0.0344],
                      [-0.0188, -0.0136,  0.0166,  ...,  0.0122,  0.0054,  0.0343],
                      ...,
                      [-0.0079,  0.0078,  0.0078,  ..., -0.0295,  0.0314,  0.0287],
                      [-0.0233,  0.0336,  0.0211,  ..., -0.0098,  0.0105,  0.0038],
                      [-0.0138, -0.0130,  0.0253,  ...,  0.0225,  0.0158,  0.0037]])),
             ('linear_relu_stack.0.bias',
              tensor([ 0.0294,  0.0148,  0.0076, -0.0061,  0.0032,  0.0023, -0.0267,  0.0253,
                       0.0117, -0.0197,  0.0110, -0.0230, -0.0116,  0.0078, -0.0291, -0.0293,
                       0.0187,  0.0368, -0.0078,  0.0149, -0.0291, -0.0277,  0.0263, -0.0027,
                       0.0327,  0.0137, -0.0033,  0.0350,  0.0359,  0.0151,  0.0226, -0.0191,
    

In [24]:
# print state_dict 
for name, param in model.state_dict().items():
    print(f"Parameter name: {name}")
    print(f"Parameter value: {param}")
    print(f"Parameter shape: {param.shape}")
    print("-" * 50)


Parameter name: linear_relu_stack.0.weight
Parameter value: tensor([[-0.0066,  0.0215,  0.0193,  ...,  0.0235,  0.0274, -0.0279],
        [-0.0350, -0.0250, -0.0346,  ..., -0.0334,  0.0346, -0.0344],
        [-0.0188, -0.0136,  0.0166,  ...,  0.0122,  0.0054,  0.0343],
        ...,
        [-0.0079,  0.0078,  0.0078,  ..., -0.0295,  0.0314,  0.0287],
        [-0.0233,  0.0336,  0.0211,  ..., -0.0098,  0.0105,  0.0038],
        [-0.0138, -0.0130,  0.0253,  ...,  0.0225,  0.0158,  0.0037]])
Parameter shape: torch.Size([512, 784])
--------------------------------------------------
Parameter name: linear_relu_stack.0.bias
Parameter value: tensor([ 0.0294,  0.0148,  0.0076, -0.0061,  0.0032,  0.0023, -0.0267,  0.0253,
         0.0117, -0.0197,  0.0110, -0.0230, -0.0116,  0.0078, -0.0291, -0.0293,
         0.0187,  0.0368, -0.0078,  0.0149, -0.0291, -0.0277,  0.0263, -0.0027,
         0.0327,  0.0137, -0.0033,  0.0350,  0.0359,  0.0151,  0.0226, -0.0191,
         0.0154,  0.0027,  0.0309,  0

In [59]:
# Access the weights and bias of the first layer

print(model.linear_relu_stack)

fc1_weights = model.linear_relu_stack[0].weight
fc1_bias = model.linear_relu_stack[0].bias

print("Weights of fc1 layer:", fc1_weights)
print("Bias of fc1 layer:", fc1_bias)


Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=10, bias=True)
)
Weights of fc1 layer: Parameter containing:
tensor([[ 0.0179, -0.0142,  0.0200,  ...,  0.0028,  0.0268,  0.0126],
        [-0.0270, -0.0023,  0.0217,  ...,  0.0134, -0.0088, -0.0113],
        [-0.0118,  0.0246,  0.0357,  ..., -0.0130, -0.0279,  0.0042],
        ...,
        [-0.0064,  0.0213, -0.0277,  ..., -0.0039, -0.0039, -0.0044],
        [ 0.0053, -0.0283, -0.0170,  ..., -0.0213,  0.0301,  0.0211],
        [-0.0285,  0.0148,  0.0307,  ..., -0.0167,  0.0261,  0.0092]],
       device='mps:0', requires_grad=True)
Bias of fc1 layer: Parameter containing:
tensor([ 3.0576e-02, -3.0341e-02,  3.1664e-02,  2.8869e-02,  2.9242e-02,
         1.4737e-02, -3.4691e-02, -1.1108e-03,  1.5957e-02,  2.7175e-02,
        -4.0652e-03,  3.0553e-02,  2.1223e-02,  3.1449e-02,  1.4513e-02

In [32]:
# print the total number of parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 932362
Trainable parameters: 932362


In [56]:
# save model state_dict
# only the model's state_dict is saved
torch.save(model.state_dict(), "model.pth")
# load model state_dict
# model must be created before loading the state_dict
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten  = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model2 = NN()


/var/folders/g0/phgwvz7x3sz18gfzz4zv9rq40000gn/T/ipykernel_57359/3466235813.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load("model.pth

NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [58]:
model2.named_parameters

<generator object Module.named_parameters at 0x3112861f0>

In [ ]:
model2.load_state_dict(torch.load("model.pth"))
# model2 is now loaded with the saved state_dict, but it is not trained
model2.eval()